# FMI Hirlam, MET Norway HARMONIE and NCEP GFS comparison demo

In this demo notebook we provide short comparison of using three different weather forecast models:
GFS -- http://data.planetos.com/datasets/noaa_gfs_pgrb2_global_forecast_recompute_0.25degree
HIRLAM -- http://data.planetos.com/datasets/fmi_hirlam_surface
HARMONIE -- http://data.planetos.com/datasets/metno_harmonie_metcoop

You can get more information about the datasets by opening links to their detail pages, but their main difference is that GFS is a global, medium range weather forecast model with lower resolution, and HIRLAM and HARMONIE are limited area models, meaning they cover only small part of the globe, but provide higher resolution of all forecasted field, in return.

First we compare the datasets by showing their spatial coverages, then we demonstrate their resolutions by showing forecast field as a discrete grid (so one can see the difference in grid cell size and resolved surface details) and finally we demonstrate plotting weather forecast for the same variable from three models. 

We try to keep this demo short, but in case you are interested in creating a more interactive notebook, please refer to our other examples:
https://github.com/planet-os/demos/blob/master/notebooks/PlanetOS_WAve_Models.ipynb
https://github.com/planet-os/notebooks/blob/master/api-examples/GFS_public_full_demo_main.ipynb

Unlike previous notebooks, we have moved most of the parsing code to external library dh_py_access, which you should get automatically if you get this notebook by cloning the git repository. 

If you have any questions, contact our team at https://data.planetos.com

At first, let's import some modules. If you do not have them, download them (ie. using pip or conda).

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import warnings
import dateutil.parser
import matplotlib
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
import xarray as xr

Import datahub parsing library

In [2]:
from dh_py_access.lib.dataset import dataset as dataset
import dh_py_access.lib.datahub as datahub
from dh_py_access import package_api

Now we define hirlam and harmonie namespaces. Add server address and our API key. 
<font color='red'>Please add your API key below:</font>

In [3]:
server = 'http://api.planetos.com/v1/datasets/'
API_key = open('APIKEY').read().strip()
print(API_key)

804e22da0c704d7b955a3d250750bc2b


In [4]:
dh=datahub.datahub_main(API_key)
fmi_hirlam_surface=dataset('fmi_hirlam_surface',dh)
metno_harmonie_metcoop=dataset('metno_harmonie_metcoop',dh)
gfs=dataset('noaa_gfs_pgrb2_global_forecast_recompute_0.25degree',dh)

One can easily see what kind of variables are available in given dataset by just calling methods:

1. long_names -- gives a long human readable name for variable, which is unfortunately not standardised in any way
2. standard_names -- gives variable names as defined in CF convention standard name table http://cfconventions.org/standard-names.html
3. variable_names -- names by which you can actually query data from the API

on a given dataset instance.

In [5]:
sample_var_names = {fmi_hirlam_surface:'Temperature_height_above_ground',
                    metno_harmonie_metcoop:'air_temperature_2m',
                    gfs:'tmp_m'}

In [18]:
fmi_hirlam_surface.get_dataset_boundaries()['coordinates']

[[[[-31.612393352985407, 25.647659301757812],
   [-31.612393352985407, 25.627549266815187],
   [-36.632966306209596, 25.627549266815187],
   [-36.632966306209596, 25.647659301757812],
   [-135.02635192871097, 25.647659301757812],
   [-135.02635192871097, 85.95765409469604],
   [-135.07557323217392, 85.95765409469604],
   [-135.07557323217392, 90],
   [180, 90],
   [180, 87.96865758895872],
   [179.989990234375, 87.96865758895872],
   [179.989990234375, 25.647659301757812],
   [42.21956184148786, 25.647659301757812],
   [42.21956184148786, 25.627549266815187],
   [32.27685854196545, 25.627549266815187],
   [32.27685854196545, 25.647659301757812],
   [-31.612393352985407, 25.647659301757812]]],
 [[[-180, 90],
   [-179.960788462162, 90],
   [-179.960788462162, 87.96865758895872],
   [-180, 87.96865758895872],
   [-180, 90]]]]

In [19]:
metno_harmonie_metcoop.get_dataset_boundaries()['coordinates']

[[[31.2984296875, 52.1929765625],
  [20.9757890625, 52.1929765625],
  [20.9757890625, 52.8953203125],
  [8.817179687500001, 52.8953203125],
  [8.817179687500001, 52.1929765625],
  [-0.5687421874999992, 52.1929765625],
  [-0.5687421874999992, 54.3000078125],
  [-1.5054609375, 54.3000078125],
  [-1.5054609375, 56.407039062500004],
  [-2.442179687500001, 56.407039062500004],
  [-2.442179687500001, 58.5140703125],
  [-3.3788984374999997, 58.5140703125],
  [-3.3788984374999997, 59.918757812500004],
  [-4.3156171875, 59.918757812500004],
  [-4.3156171875, 61.32344531249999],
  [-5.2523359375, 61.32344531249999],
  [-5.2523359375, 62.72813281250001],
  [-6.1890546875, 62.72813281250001],
  [-6.1890546875, 64.13282031250002],
  [-7.1257734375, 64.13282031250002],
  [-7.1257734375, 65.53750781250001],
  [-8.062492187500002, 65.53750781250001],
  [-8.062492187500002, 66.23985156250001],
  [-8.999210937500001, 66.23985156250001],
  [-8.999210937500001, 66.94219531250002],
  [-9.9359296875, 66.942

In [14]:
def get_max_coverage_package(dataset, area_name, varfilter = 'temp'):
    coords = dataset.get_dataset_boundaries()['coordinates']
    ds_west = np.amin([i[0] for i in coords[0]])
    ds_east = np.amax([i[0] for i in coords[0]])
    ds_south = np.amin([i[1] for i in coords[0]])
    ds_north = np.amax([i[1] for i in coords[0]])
    print("Bounds test: ", dataset.datasetkey, ds_west, ds_east, ds_south, ds_north)
    ## temperature_variable = [i for i in dataset.variable_names() if varfilter in i.lower()][0]
    temperature_variable = sample_var_names[dataset]
    assert len(temperature_variable) >= 1, "something wrong {0}".format(temperature_variable)
    assert type(temperature_variable) == str
    return package_api.package_api(dh,dataset.datasetkey,temperature_variable,ds_west,ds_east,ds_south,ds_north,area_name=area_name)

In [15]:
area_name = 'maximum_03'

In [16]:
package_harmonie = get_max_coverage_package(metno_harmonie_metcoop, area_name=area_name)
package_fmi_hirlam = get_max_coverage_package(fmi_hirlam_surface, area_name=area_name)

Bounds test:  metno_harmonie_metcoop -11.809367187500001 48.1593671875 52.1929765625 73.97967968750001
Bounds test:  fmi_hirlam_surface -31.612393352985407 25.647659301757812 -31.612393352985407 25.627549266815187


In [12]:
package_harmonie.make_package()
package_fmi_hirlam.make_package()

Package request: http://api.planetos.com/v1/packages?apikey=804e22da0c704d7b955a3d250750bc2b&dataset=metno_harmonie_metcoop&polygon=[[-11.809367187500001,52.1929765625],[48.1593671875,52.1929765625],[48.1593671875,73.97967968750001],[-11.809367187500001,73.97967968750001],[-11.809367187500001,52.1929765625]]&grouping=location&package=metno_harmonie_metcoop_recent_reftime_20180118_maximum_03&var=air_temperature_2m
Package request: http://api.planetos.com/v1/packages?apikey=804e22da0c704d7b955a3d250750bc2b&dataset=fmi_hirlam_surface&polygon=[[-31.612393352985407,-31.612393352985407],[25.647659301757812,-31.612393352985407],[25.647659301757812,25.627549266815187],[-31.612393352985407,25.627549266815187],[-31.612393352985407,-31.612393352985407]]&grouping=location&package=fmi_hirlam_surface_recent_reftime_20180118_maximum_03&var=Temperature_height_above_ground


In [13]:
package_harmonie.download_package()
package_fmi_hirlam.download_package()

ValueError: Package creation failed

In [ ]:
data_harmonie = xr.open_dataset(package_harmonie.get_local_file_name())
data_fmi_hirlam = xr.open_dataset(package_fmi_hirlam.get_local_file_name(),decode_cf=False)

Take GFS for area of HARMONIE

In [ ]:
np.amin(data_harmonie['lon'].data)

In [ ]:
left = np.amin(data_harmonie['lon'].data)
right = np.amax(data_harmonie['lon'].data)
bottom = np.amin(data_harmonie['lat'].data)
top = np.amax(data_harmonie['lat'].data)

gfs_temperature_var = [i for i in gfs.variable_names() if 'temp' in i.lower()][0]
package_gfs = package_api.package_api(dh,gfs.datasetkey,gfs_temperature_var,left,right,bottom,top,area_name=area_name)
package_gfs.make_package()
package_gfs.download_package()

In [ ]:
data_gfs = xr.open_dataset(package_gfs.get_local_file_name(),decode_cf=False)

In [ ]:
[i for i in data_gfs.variables]

In [ ]:
plt.pcolormesh(data_harmonie['air_temperature_2m'][0,0,:,:])

In [ ]:
data_gfs.variables

## Dataset extent and resolution

Get some arbitrary field for demonstration, we use 2m temperature and as you can see, variable names may actually differ a lot between datasets. Please note that "get_tds_field" method is just for getting arbitrary preview image, if you wan't to query data for specific time and reftime, please refer to examples for our raster API (shown in other notebooks referenced to above) or use THREDDS server link given in dataset detail pages.

In [ ]:


hirdat=fmi_hirlam_surface.get_tds_field(sample_var_names[fmi_hirlam_surface])
harmoniedat=metno_harmonie_metcoop.get_tds_field(sample_var_names[metno_harmonie_metcoop])
harmoniedat=np.ma.masked_where(np.isnan(harmoniedat),harmoniedat)
gfsdat=gfs.get_tds_field(sample_var_names[gfs])

### Extent
The easiest way to show dataset extent is to plot it on a map with proper projection. We do not show GFS here, because, well, it is global.

In [ ]:
m = Basemap(projection='ortho',lon_0=10,lat_0=50,resolution='l')
hir_x,hir_y=np.meshgrid(fmi_hirlam_surface.get_tds_field('lon'),fmi_hirlam_surface.get_tds_field('lat'))
X,Y=m(hir_x,hir_y)
fig=plt.figure()
plt.subplot(221)
m.pcolormesh(X,Y,hirdat)
m.drawcoastlines()
plt.subplot(222)
harm_x,harm_y=np.meshgrid(metno_harmonie_metcoop.get_tds_field('lon'),metno_harmonie_metcoop.get_tds_field('lat'))
X,Y=m(harm_x,harm_y)
m.pcolormesh(X,Y,harmoniedat)
m.drawcoastlines()
plt.colorbar()

### Resolution 

Let's zoom in a little to illustrate difference in resolutions. By plotting the gridded data as a mesh, one can easily get the grid size from the figures. Plot's given for the Norwegian coast.

In [ ]:
m2 = Basemap(projection='merc',llcrnrlat=58,urcrnrlat=59,\
            llcrnrlon=5,urcrnrlon=7,lat_ts=58,resolution='i')

In [ ]:
gfs_x,gfs_y=np.meshgrid(gfs.get_tds_field('lon'),gfs.get_tds_field('lat'))
fig=plt.figure(figsize=(8,8))
plt.subplot(221)
X,Y=m2(hir_x,hir_y)
x1,y1,x2,y2=470,2055,493,2090
m2.pcolormesh(X[x1:x2,y1:y2],Y[x1:x2,y1:y2],hirdat[x1:x2,y1:y2])
m2.drawcoastlines()
plt.colorbar()
plt.subplot(222)
X,Y=m2(harm_x,harm_y)
x1,y1,x2,y2=230,330,275,380
m2.pcolormesh(X[x1:x2,y1:y2],Y[x1:x2,y1:y2],harmoniedat[x1:x2,y1:y2])
m2.drawcoastlines()
plt.colorbar()
plt.subplot(223)
X,Y=m2(gfs_x,gfs_y)
x1,y1,x2,y2=124,20,130,29
m2.pcolormesh(X[x1:x2,y1:y2],Y[x1:x2,y1:y2],gfsdat[x1:x2,y1:y2])
m2.drawcoastlines()
plt.colorbar()

Can you guess which model is on which map by just looking at these images?

### Forecast for a single location
First, get point data for all datasets for given variable and for as long time range as the forecast goes.

In [ ]:
sample_point_data = [(k,k.get_json_data_in_pandas(**{'var':v,'lon':longitude,'lat':latitude,'count':1000})) for k,v in sample_var_names.items()]

In [ ]:
fig = plt.figure(figsize=(11,6))
for ddd in sample_point_data:
    zlevels = [2.]
    for i in zlevels:
        pdata=np.array(ddd[1][ddd[1]['z']==i][sample_var_names[ddd[0]]],dtype=np.float)
        if np.sum(np.isnan(pdata))!=pdata.shape[0]:
            plt.plot(ddd[1][ddd[1]['z']==i]['time'].apply(dateutil.parser.parse),pdata,label=str(i) + "_" + ddd[0].datasetkey)
plt.legend()
plt.grid()
fig.autofmt_xdate()
plt.title('2m temperature forecast in different weather models')
plt.show()